In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np

In [31]:
class AirModelGRU(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super().__init__()
        self.gru = nn.GRU(input_size=3, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.linear(x)
        return x

In [32]:
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i:i + lookback]
        target = dataset[i + 1:i + lookback + 1]
        X.append(feature)
        y.append(target)
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return torch.tensor(X), torch.tensor(y)

train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')

lookback = 3
X_train, y_train = create_dataset(train_data, lookback)
X_test, y_test = create_dataset(test_data, lookback)


train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')

In [33]:
hidden_size = 100
num_layers = 2
n_epochs = 10

In [34]:
print(X_train.shape)
print(X_test.shape)

torch.Size([928, 3])
torch.Size([456, 3])


In [36]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = AirModelGRU(hidden_size, num_layers).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

loader = DataLoader(TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

for epoch in tqdm(range(n_epochs), desc="Training epochs"):
    model.train()
    train_loss = 0
    for batch in loader:
        X, y = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X.size(0)
    
    train_loss /= len(loader.dataset)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

torch.save(model.state_dict(), 'GRU.pth')

model.eval()
with torch.no_grad():
    predictions = model(X_test.to(device))
    print("Predictions:", predictions.cpu().numpy())

Training epochs:  20%|██        | 2/10 [00:00<00:01,  5.73it/s]

Epoch 1, Train Loss: 0.0159
Epoch 2, Train Loss: 0.0063


Training epochs:  40%|████      | 4/10 [00:00<00:00,  6.03it/s]

Epoch 3, Train Loss: 0.0060
Epoch 4, Train Loss: 0.0059


Training epochs:  60%|██████    | 6/10 [00:01<00:00,  6.00it/s]

Epoch 5, Train Loss: 0.0058
Epoch 6, Train Loss: 0.0058


Training epochs:  80%|████████  | 8/10 [00:01<00:00,  6.15it/s]

Epoch 7, Train Loss: 0.0057
Epoch 8, Train Loss: 0.0057


Training epochs: 100%|██████████| 10/10 [00:01<00:00,  6.05it/s]

Epoch 9, Train Loss: 0.0058
Epoch 10, Train Loss: 0.0059
Predictions: [[0.16595298]
 [0.14492118]
 [0.11124821]
 [0.12755835]
 [0.15140331]
 [0.14904061]
 [0.17407887]
 [0.16580996]
 [0.10938173]
 [0.06700751]
 [0.07456356]
 [0.08974874]
 [0.06919913]
 [0.08472256]
 [0.13997686]
 [0.14995064]
 [0.10490574]
 [0.08587183]
 [0.11056449]
 [0.1301845 ]
 [0.13481647]
 [0.1402328 ]
 [0.11844417]
 [0.12426402]
 [0.12407767]
 [0.09012716]
 [0.09185787]
 [0.1560015 ]
 [0.20876357]
 [0.1891048 ]
 [0.16098657]
 [0.18274081]
 [0.19925293]
 [0.18645525]
 [0.18797582]
 [0.20766875]
 [0.19699821]
 [0.13731655]
 [0.14673758]
 [0.21116811]
 [0.14989263]
 [0.14615059]
 [0.2612579 ]
 [0.34810373]
 [0.39104864]
 [0.22278059]
 [0.22204852]
 [0.25416723]
 [0.36890146]
 [0.38653302]
 [0.3243063 ]
 [0.4169311 ]
 [0.5825616 ]
 [0.50392884]
 [0.22165775]
 [0.08309871]
 [0.05449853]
 [0.07802507]
 [0.07881817]
 [0.07664399]
 [0.07307972]
 [0.08998635]
 [0.08115945]
 [0.08084051]
 [0.11488898]
 [0.12285519]
 [0.11